## 深度学习自然语言处理第二次作业

##### 李明昕 SY2206124

### 1. EM 算法原理

EM 算法用于估计既含有观测变量又含有隐变量的概率模型的参数，其原理如下$^{[1]}$：

假设观测变量数据为 $Y$ ，隐变量数据为 $Z$ ，他们的联合分布为 $P(Y,Z|\theta)$ ，条件分布为 $P(Z|Y,\theta)$ ，其中 $\theta$ 为模型参数。EM 算法的过程如下：

1. 选择参数的初值 $\theta^{(0)}$ ，开始迭代；
2. E 步：记 $\theta^{(i)}$ 为第 $i$ 次迭代参数 $\theta$ 的估计值，在第 $i + 1$ 次迭代的 E 步，计算
   $$
    \begin{aligned}
    Q(\theta,\theta^{(i)}) &= E_Z[\log P(Y,Z|\theta)|Y,\theta^{(i)}]\\
    &= \sum_Z \log P(Y,Z|\theta)P(Z|Y,\theta^{(i)}).
    \end{aligned}
   $$
   这里 $P(Z|Y,\theta^{(i)})$ 是在给定观测数据 $Y$ 和当前的参数估计 $\theta^{(i)}$ 下隐变量 $Z$ 的条件概率分布；
3. M 步：求使 $Q(\theta,\theta^{(i)})$ 极大化的 $\theta$ ，确定第 $i + 1$ 次迭代的参数估计值 $\theta^{(i + 1)}$
   $$
    \theta^{(i + 1)} = \arg\max_\theta Q(\theta,\theta^{(i)});
   $$ 
4. 重复 2、3 步，直到收敛。

### 2. 身高高斯混合模型参数估计

#### 2.1 模型定义   

身高的概率模型假设为一种高斯混合分布，其中观测变量 $y$ 为身高（单位 cm），隐变量 $z$ 为性别（$z=0$ 时表示男生， $z=1$ 时表示女生）。概率模型表示为：

+ 表示男生身高的高斯分布：
  $$
    P(y|z=0,\theta) = N(y|\mu_0,\sigma_0^2)
  $$
+ 表示女生身高的高斯分布：
  $$
    P(y|z=1,\theta) = N(y|\mu_1,\sigma_1^2)
  $$
+ 表示性别的分布：
  $$
    P(z) = \begin{cases}
    \pi & z=0\\
    1-\pi & z=1
    \end{cases}
  $$
+ 则表示身高的高斯混合分布为：
  $$
    P(y) = \pi N(y|\mu_0,\sigma_0^2) + (1-\pi) N(y|\mu_1,\sigma_1^2)
  $$
  参数为 $\theta = \{\mu_0,\sigma_0^2,\mu_1,\sigma_1^2,\pi\}$ 。

#### 2.2 EM 算法

假设观测变量数据为：$Y=\{y_1,y_2,\cdots,y_n\}$ ，隐变量观测数据为：$Z=\{z_1,z_2,\cdots,z_n\}$，其中 $y_i, z_i$ 独立同分布，则：
1. E 步：

   $$
    \begin{aligned}
      Q(\theta,\theta^{(i)}) =& \sum_Z\log P(Y,Z|\theta)P(Z|Y,\theta^{(i)}) \\
      =&\sum_Z\log \prod_{j=1}^nP(y_j,z_j|\theta)P(z_j|y_j,\theta^{(i)}) \\
      =&\sum_Z\sum_{j=1}^n\log P(y_j,z_j|\theta)P(z_j|y_j,\theta^{(i)}) \\
      =&2^{n-1}\sum_{j=1}\log\left(P(y_j,z_j=0|\theta)P(z_j=0|y_i,\theta^{(i)}) + P(y_j,z_j=1|\theta)P(z_j=1|y_j,\theta^{(i)})\right) \\
      =&2^{n-1}\sum_{j=1}\log\Big(\pi N(y_j|\mu_0,\sigma_0^2)\frac{\pi^{(i)}N(y_j|\mu_0^{(i)}, \sigma_0^{(i)})}{\pi^{(i)}N(y_j|\mu_0^{(i)}, \sigma_0^{(i)}) + (1-\pi^{(i)})N(y_j|\mu_1^{(i)}, \sigma_1^{(i)})} \\
      &+ (1-\pi)N(y_j|\mu_1,\sigma_1^2)\frac{(1-\pi^{(i)})N(y_j|\mu_1^{(i)}, \sigma_1^{(i)})}{\pi^{(i)}N(y_j|\mu_0^{(i)}, \sigma_0^{(i)}) + (1-\pi^{(i)})N(y_j|\mu_1^{(i)}, \sigma_1^{(i)})}\Big) \\
    \end{aligned}
   $$
   
   记 $\log$ 内的式子为 $A_j$ 。
2. M 步：通过极大似然估计求解参数 $\theta$ ，得到参数估计值 $\theta^{(i+1)}$ 。

   首先需要计算：$\gamma_j^{(i)} = P(z_j=0|y_j, \theta^{(i)}) = \frac{\pi^{(i)}N(y_j|\mu_0^{(i)}, \sigma_0^{(i)})}{\pi^{(i)}N(y_j|\mu_0^{(i)}, \sigma_0^{(i)}) + (1-\pi^{(i)})N(y_j|\mu_1^{(i)}, \sigma_1^{(i)})}$ 。

   然后每个参数的估计值如下：

   + $\pi^{(i+1)} = \frac{1}{n}\sum_{j=1}^n\gamma_j^{(i)}$ ；
   + $\mu_0^{(i+1)} = \frac{\sum_{j=1}^n\gamma_j^{(i)}y_j}{\sum_{j=1}^n\gamma_j^{(i)}}$ ；
   + $\mu_1^{(i+1)} = \frac{\sum_{j=1}^n(1-\gamma_j^{(i)})y_j}{\sum_{j=1}^n(1-\gamma_j^{(i)})}$ ；
   + $(\sigma_0^{(i+1)})^2 = \frac{\sum_{j=1}^n\gamma_j^{(i)}(y_j-\mu_0^{(i)})^2}{\sum_{j=1}^n\gamma_j^{(i)}}$ ；
   + $(\sigma_1^{(i+1)})^2 = \frac{\sum_{j=1}^n(1-\gamma_j^{(i)})(y_j-\mu_1^{(i)})^2}{\sum_{j=1}^n(1-\gamma_j^{(i)})}$ 。

3. 重复 2、3 步，直到参数估计值的变化小于某个阈值。

#### 2.3 代码实现

In [68]:
import pdb
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm
from tqdm import tqdm
from typing import List, Dict

##### 2.3.1 读取数据

In [45]:
DATA_PATH = r'src\height_data.csv'
data = pd.read_csv(DATA_PATH)

##### 2.3.2 计算 $\gamma_j$（E 步）

In [58]:
def E_step(theta: Dict[str, float], data: pd.DataFrame) -> List[float]:
    gammas = []
    for i in range(len(data)):
        gamma0 = theta['pi'] * norm.pdf(data['height'][i], loc=theta['mu_0'], scale=theta['sigma2_0']**(1/2))
        gamma1 = (1 - theta['pi']) * norm.pdf(data['height'][i], loc=theta['mu_1'], scale=theta['sigma2_1']**(1/2))
        gammas.append(gamma0 / (gamma0 + gamma1))
    return gammas

##### 2.3.3 计算新的参数估计值（M 步）

In [53]:
def M_step(theta: Dict[str, float], gammas: List[float], data: pd.DataFrame):
    new_pi = sum(gammas) / len(gammas)
    new_mu_0 = sum(gammas[i] * data['height'][i] for i in range(len(gammas))) / sum(gammas)
    new_mu_1 = sum((1 - gammas[i]) * data['height'][i] for i in range(len(gammas))) / (len(gammas) - sum(gammas))
    new_sigma2_0 = sum(gammas[i] * (data['height'][i] - theta['mu_0']) ** 2 for i in range(len(gammas))) / sum(gammas)
    new_sigma2_1 = sum((1 - gammas[i]) * (data['height'][i] - theta['mu_1']) ** 2 for i in range(len(data))) / (len(data) - sum(gammas))
    return {
        'pi': new_pi,
        'mu_0': new_mu_0,
        'mu_1': new_mu_1,
        'sigma2_0': new_sigma2_0,
        'sigma2_1': new_sigma2_1
    }

##### 2.3.4 迭代估计参数值

In [56]:
def EM(theta_0: Dict[str, float], data: pd.DataFrame, threshold: float = 1e-5):
    thetas = [theta_0]
    while True:
        pre_theta = thetas[-1]
        gammas = E_step(pre_theta, data)
        new_theta = M_step(pre_theta, gammas, data)
        thetas.append(new_theta)
        if max(abs(new_theta[key] - pre_theta[key]) for key in new_theta) < threshold:
            break
    return thetas

### 实验

从生成数据的代码中可以看出，各个参数的真实值为：
+ $\pi^{\mathrm{ture}} = 0.75$
+ $\mu_O^{\mathrm{ture}} = 176$
+ $\mu_1^{\mathrm{ture}} = 164$
+ $(\sigma_0^{\mathrm{ture}})^2 = 25$
+ $(\sigma_1^{\mathrm{ture}})^2 = 9$


In [62]:
theta_true = {
    'pi': 0.75,
    'mu_0': 176,
    'mu_1': 165,
    'sigma2_0': 25,
    'sigma2_1': 9
}

#### 根据先验设定初始参数值

In [60]:
# 根据官方数据，我国的男女比例以及男女身高的均值和标准差为
theta_0 = {
    'pi': 0.5169,
    'mu_0': 169.7,
    'mu_1': 158.,
    'sigma2_0': 10,
    'sigma2_1': 10
}

thetas = EM(theta_0, data, threshold=1e-5)
print(f'iteration num: {len(thetas)}')
for p, v in thetas[-1].items():
    print(f'{p}: {v:.4f}')

iteration num: 134
pi: 0.7500
mu_0: 176.2608
mu_1: 163.9262
sigma2_0: 23.5517
sigma2_1: 8.6329


##### 平均相对误差

In [64]:
# 平均相对误差为
print(f'average relative error: {sum(abs(thetas[-1][key] - theta_true[key]) / theta_true[key] for key in thetas[-1]) / len(thetas[-1])}')

average relative error: 0.02134774314920935


所以使用我国官方的数据进行参数初始化，经过 EM 算法后的结果为：

| 参数 | 估计值 | 真实值 |
| --- | --- | --- |
| $\pi$ | 0.75 | 0.75 |
| $\mu_0$ | 176.26 | 176.00 |
| $\mu_1$ | 164.93 | 164.00 |
| $(\sigma_0)^2$ | 23.55 | 25.00 |
| $(\sigma_1)^2$ | 8.63 | 9.00 |

平均相对误差为：0.02135

##### 实验结果可视化

![experiment_1](resources/experiment_1.png)

#### 在给定范围内初始化参数

由于 EM 算法对初始值敏感，所以在给定范围内初始化多组参数，然后进行 EM 算法，以此来验证 EM 算法的稳定性。

In [73]:
# 不同的参数初始化
pis = np.linspace(0.1, 0.9, 10)
mus_0 = np.linspace(170., 180., 10)
mus_1 = np.linspace(155., 165., 10)
sigma2s_0 = np.linspace(5., 15., 10)
sigma2s_1 = np.linspace(5., 15., 10)
init_thetas = [{'pi': pi, 'mu_0': mu_0, 'mu_1': mu_1, 'sigma2_0': sigma2_0, 'sigma2_1': sigma2_1} 
                for pi, mu_0, mu_1, sigma2_0, sigma2_1 in zip(pis, mus_0, mus_1, sigma2s_0, sigma2s_1)]

# 从不同的初始值开始 EM 算法，并输出结果
for theta_0 in init_thetas:
    thetas = EM(theta_0, data, threshold=1e-5)
    theta = thetas[-1]
    print(f'init theta: {theta_0}')
    print(f'iteration num: {len(thetas)}')
    for p, v in theta.items():
        print(f'{p}: {v:.4f}')
    print(f'average relative error: {sum(abs(theta[key] - theta_true[key]) / theta_true[key] for key in theta) / len(theta)}')
    print('-' * 20)

init theta: {'pi': 0.1, 'mu_0': 170.0, 'mu_1': 155.0, 'sigma2_0': 5.0, 'sigma2_1': 5.0}
iteration num: 137
pi: 0.7500
mu_0: 176.2608
mu_1: 163.9262
sigma2_0: 23.5517
sigma2_1: 8.6329
average relative error: 0.02134772532970382
--------------------
init theta: {'pi': 0.18888888888888888, 'mu_0': 171.11111111111111, 'mu_1': 156.11111111111111, 'sigma2_0': 6.111111111111111, 'sigma2_1': 6.111111111111111}
iteration num: 132
pi: 0.7500
mu_0: 176.2608
mu_1: 163.9262
sigma2_0: 23.5517
sigma2_1: 8.6329
average relative error: 0.02134772411528298
--------------------
init theta: {'pi': 0.2777777777777778, 'mu_0': 172.22222222222223, 'mu_1': 157.22222222222223, 'sigma2_0': 7.222222222222222, 'sigma2_1': 7.222222222222222}
iteration num: 125
pi: 0.7500
mu_0: 176.2608
mu_1: 163.9262
sigma2_0: 23.5517
sigma2_1: 8.6329
average relative error: 0.021347748634670115
--------------------
init theta: {'pi': 0.3666666666666667, 'mu_0': 173.33333333333334, 'mu_1': 158.33333333333334, 'sigma2_0': 8.3333333

可以看到模型在不同参数初始化下也能收敛到相近的结果

### 参考文献

[1] 李航.统计学习方法.北京:清华大学出版社,2012:175-187.